In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#Import the data and drop unnamed:0 column
train = pd.read_csv(r'D:\Subodh\Projects\Credit Risk\Calculated PD\Data\cs-training_1.csv')
test = pd.read_csv(r'D:\Subodh\Projects\Credit Risk\Calculated PD\Data\cs-test_1.csv')
train = train.drop(['Unnamed: 0'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)
train.head()
test.head()
train.shape

test.shape
#Find duplicates
train.duplicated().value_counts()
#Drop duplicates
train_redup = train.drop_duplicates()
#Checking if duplicates still persist
train_redup.duplicated().sum()
#Check for null 
train_redup.isnull().sum()
# % of null values
round(train_redup.isnull().sum()/train_redup.shape[0]*100,2)
def findMiss(df):
    return round(df.isnull().sum()/df.shape[0]*100,2)

findMiss(train_redup)
#Filtering data where null values are present for no. of dependents
train_redup[train_redup.NumberOfDependents.isnull()]

train_redup[train_redup.NumberOfDependents.isnull()].describe()
#We observe that monthly income is not present wherein no. of dependents data is null
train_redup[train_redup.MonthlyIncome.isnull()].describe()
#We observe that maximum no of depedents are 9 where monthyly income is null
train_redup['NumberOfDependents'].agg(['mode'])
#this means most of the rows have zero dependendts
train_redup.groupby(['NumberOfDependents']).size()
#Create a subset of dataset and fill null as 0 
fam_miss = train_redup[train_redup.NumberOfDependents.isnull()]
fam_nmiss = train_redup[train_redup.NumberOfDependents.notnull()]
 
fam_miss.shape

fam_nmiss.shape
fam_miss['NumberOfDependents'] = fam_miss['NumberOfDependents'].fillna(0)
fam_miss['MonthlyIncome'] =  fam_miss['MonthlyIncome'].fillna(0)
#recheck for missing values
findMiss(fam_miss)
findMiss(fam_nmiss)
#evaluate

fam_nmiss['MonthlyIncome'].agg(['mean','median', 'min'])

#We observe that mean and median are not close to each other. Also mean value is 0. Hence, not a good idea to fill null with min values. 
fam_nmiss['MonthlyIncome'].agg(['max'])

#We can conclude that there are definitely some outliers in the data. Hence, we would go forward with the median values. 
fam_nmiss['MonthlyIncome'] = fam_nmiss['MonthlyIncome'].fillna(fam_nmiss['MonthlyIncome'].median())
#Check for missing values
findMiss(fam_nmiss)
#Concat both the dataset
filled_train = pd.concat([fam_nmiss, fam_miss], axis=0)

filled_train.shape
#Check for missing values in concat datset
findMiss(filled_train)
filled_train.head()
#Finding the proportion of delinq and non-delinq customers
filled_train.groupby(['SeriousDlqin2yrs']).size()/filled_train.shape[0]
#93% are non-delinq and approx 6.7% are delinq

#Biased data

#Hence analyze the data on each variable and figure out which rows can be removed from the data for clean analysis 
filled_train.RevolvingUtilizationOfUnsecuredLines.describe()
filled_train['RevolvingUtilizationOfUnsecuredLines'].quantile([.5])
filled_train['RevolvingUtilizationOfUnsecuredLines'].quantile([.9])
filled_train['RevolvingUtilizationOfUnsecuredLines'].quantile([.95])
filled_train['RevolvingUtilizationOfUnsecuredLines'].quantile([.99])
# Upto 99th percentile data lies between 0 and 1. Clearly there are some outliers in the data 
#Let's divide the data and analyze

filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] < 1].describe()

#146053 records are less than 1
filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 1].describe()
# 3321 records have values greater than 1
# avg default rate is 0.372478

filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 1].describe()

filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 10].describe().groupby(['SeriousDlqin2yrs']).size()
# 241 records have values greater than 10
# avg default rate is 0.070539
filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 10].groupby(['SeriousDlqin2yrs']).size()
#out of 241, only 17 are defaulters
filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 1].groupby(['SeriousDlqin2yrs']).size()

#We cannot simply remove the data where the value is greater than 1. It will make the data more imbalanced
#we will remove the values greater than 10 considering them as outliers
#Ideally we should not consider values more than 1 but after analyzing the data we might miss some important info

#Checking for impact on other variables
filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 10].describe()

until_dropped = filled_train.drop(filled_train[filled_train['RevolvingUtilizationOfUnsecuredLines'] > 10].index)

until_dropped.shape

#records came down from 149391
until_dropped.head()
#Checking for outliers in age with boxplot

sns.boxplot(data=until_dropped, x='age')

#We can see the outlier where age is zero which is not possible 

#We can go ahead and remove this. We can revert while training the model
until_dropped.groupby(['NumberOfTime30-59DaysPastDueNotWorse']).size()

#No values between 13 and 96
#We can observe the outliers where customers have delayed the payments 96,98 times but not for 14 or 15 times
#Looks like an error in data but let's check for no. of defaulters

#Let's analze all the delayed payments

until_dropped.groupby(['NumberOfTime60-89DaysPastDueNotWorse']).size()

until_dropped.groupby(['NumberOfTimes90DaysLate']).size()


#After 17 times the nearest number is 96 in all the three dataset
#Checking for impact 
until_dropped[until_dropped['NumberOfTimes90DaysLate'] >=96 ]['SeriousDlqin2yrs'].describe()
#About 60% are delinq
until_dropped[until_dropped['NumberOfTimes90DaysLate'] >=96 ].groupby(['SeriousDlqin2yrs']).size()
#There a good chunk of values greater than 1 hence would not be good idea to delete this. let's check for other ways to treat this data
#We will try to winsorizing. Treating outliers to come closer to the other values
#Pending variable = debt ratio

until_dropped['DebtRatio'].describe()

sns.kdeplot(until_dropped['DebtRatio'])

#most of the points between 0 and 50000
#finding the proportion of outliers

until_dropped['DebtRatio'].quantile([.9])

until_dropped['DebtRatio'].quantile([.5])

until_dropped['DebtRatio'].quantile([.75])

until_dropped['DebtRatio'].quantile([.95])

until_dropped['DebtRatio'].quantile([.975])

until_dropped[until_dropped['DebtRatio']>3000].describe()
#There are 5226 records have values more than 3000
until_dropped[until_dropped['DebtRatio']>3500].describe()

#Checking for delinq accounts

#3492 is taken from 97.5 percentile

until_dropped[until_dropped['DebtRatio']>3492].groupby(['SeriousDlqin2yrs']).size()

until_dropped[until_dropped['DebtRatio']>3492].describe()

until_dropped[until_dropped['DebtRatio']>3492][['SeriousDlqin2yrs', 'MonthlyIncome']].describe()


#checking if for some of the records if monthly income and target variable are same(seriousdelin2yers)

until_dropped[(until_dropped['DebtRatio'] >3492 ) & (until_dropped['SeriousDlqin2yrs'] == until_dropped['MonthlyIncome'])].describe()


#Checking for delinq accounts

temp = until_dropped[(until_dropped['DebtRatio'] >3492 ) & (until_dropped['SeriousDlqin2yrs'] == until_dropped['MonthlyIncome'])]

temp.head()
temp.groupby(['SeriousDlqin2yrs']).size()

#We can observe only 2 counts greater than 1 i.e delinq accounts. We can go ahead and remove the 382 values
dRatio = until_dropped.drop(until_dropped[(until_dropped['DebtRatio']>3492) & (until_dropped['SeriousDlqin2yrs']==until_dropped['MonthlyIncome'])].index)

dRatio.shape
dRatio.head()

#We have evaluated all the variables. We can ignore NumberRealEstateLoansOrLines
until_dropped.shape
#Checking if we have dropped the variables
dRatio.shape
dRatio.groupby('SeriousDlqin2yrs').size()/dRatio.shape[0]
#We still have data imbalance hence we will go with xgboost
pip install Xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
model = XGBClassifier(tree_method = 'exact')
x = dRatio.drop(['SeriousDlqin2yrs'], axis=1) #dependent variable

x.head()
y = dRatio['SeriousDlqin2yrs'] #independent variable

model.fit(x,y.values.ravel())

y_pred = model.predict(x)
accuracy_score(y,y_pred)

#we are getting the low accuracy as data is not perfectly balanced
confusion_matrix(y,y_pred)
#true negative = 137966. Not defaulters

cm = confusion_matrix(y,y_pred)

sns.heatmap(cm,annot=True,fmt='d',cmap='Oranges',linewidths=0.5,linecolor='Black')
plt.xticks(np.arange(2)+.5,['No def','def'])
plt.yticks(np.arange(2)+.5,['No def','def'])
plt.xlabel("predicted")
plt.ylabel("actuals")

# 7144 are false negatives 
print(classification_report(y,y_pred))
#For the "No default" class (SeriousDlqin2yrs = 0), the precision is high, approximately 95%. This indicates that when the model predicts "No default," it is correct about 95% of the time.
#For the "Default" class (SeriousDlqin2yrs = 1), the precision is lower, about 78%. This means that when the model predicts "Default," it is correct about 78% of the time.
#For the "No default" class (SeriousDlqin2yrs = 0), the recall is high, approximately 99%. This indicates that the model correctly identifies about 99% of the actual "No default" cases.
#For the "Default" class (SeriousDlqin2yrs = 1), the recall is lower, about 28%. This means that the model identifies only about 28% of the actual "Default" cases.
#The F1-score is a balance between precision and recall. For the "No default" class, it's approximately 97%, indicating good overall performance.
#For the "Default" class, it's much lower, about 42%, indicating that achieving a balance between precision and recall for this class is challenging.
#The overall accuracy of the model is approximately 95%, which looks impressive at first glance. 
#However, it's important to consider the class imbalance in the dataset, where the majority of cases are "No default."
#The macro average of precision, recall, and F1-score is calculated by taking the average of these metrics for both classes, giving equal weight to each class. It is approximately 0.86.
#The weighted average considers class imbalance and is approximately 0.94, indicating the overall weighted performance across both classes.
#In summary, the model performs well in correctly identifying non-default cases (SeriousDlqin2yrs = 0) but struggles with identifying default cases (SeriousDlqin2yrs = 1), likely due to the class imbalance. 
#Further efforts to balance the dataset or explore different modeling techniques may improve the model's performance for default prediction.